In [1]:
from openpose import OpenPose
from data import Data
import os
import glob
import csv
import numpy as np
import misvm
import random
from sklearn.grid_search import GridSearchCV
# activate py2

/home/junkado/.pyenv/versions/anaconda3-2.2.0/envs/py2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/junkado/.pyenv/versions/anaconda3-2.2.0/envs/py2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# read hard
with open('2d-data/video_dir_info.txt', 'rb') as f:
    videosdir = f.readline()

with open('hard-video.csv', 'r') as f:
    hard_csvfiles = f.read().split('\n')[:-1]

with open('easy-video.csv', 'r') as f:
    easy_csvfiles = f.read().split('\n')[:-1]

hard_csvfiles = ['C' + file + '.MP4' for file in hard_csvfiles]
easy_csvfiles = ['C' + file + '.MP4' for file in easy_csvfiles]

csvfiles = sorted(glob.glob('2d-data/*.csv'))

bags = []  # tuple [instance(time), feature vector]
labels = []
for index, csvfile in enumerate(csvfiles):
    with open(csvfile, 'rb') as f:
        reader = csv.reader(f)
        videopath = next(reader)[0]
        videoname = videopath.split('/')
        videoname = videoname[-1]

        header = next(reader)

        time_row = [row for row in reader]

        data = Data(videopath, header[1], header[3], header[5], header[7], time_row)
        if not data.norm():
            continue
        """
        x, y, c, times = data.elim_nan()

        if videoname in hard_csvfiles:
            bags.append(np.array([[x[int(i / 2)][time] if i % 2 == 0 else y[int(i / 2)][time] for i in range(len(x) * 2)] for time in times]))
            labels.append(1)

        elif videoname in easy_csvfiles:
            bags.append(np.array([[x[int(i / 2)][time] if i % 2 == 0 else y[int(i / 2)][time] for i in range(len(x) * 2)] for time in times]))
            labels.append(-1)
        """
        times = data.elim_nan()

        if videoname in hard_csvfiles:
            bags.append(np.array(
                [[data.x[int(i / 2)][time] if i % 2 == 0 else data.y[int(i / 2)][time] for i in range(len(data.x) * 2)]
                 for time in times]))
            labels.append(1)

        elif videoname in easy_csvfiles:
            bags.append(np.array(
                [[data.x[int(i / 2)][time] if i % 2 == 0 else data.y[int(i / 2)][time] for i in range(len(data.x) * 2)]
                 for time in times]))
            labels.append(-1)

labels = np.array(labels, dtype=float)

In [3]:
clf = misvm.miSVM(max_iters=10, verbose=True)
tuned_parameters = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    # {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]},
    # {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree': [2, 3, 4], 'gamma': [0.001, 0.0001]},
    # {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.001, 0.0001]}
]
grid_search = GridSearchCV(clf, tuned_parameters, cv=5, scoring='f1_weighted', verbose=True)
grid_search.fit(bags, labels)
print("score")
print grid_search.grid_scores_
print "best"
print grid_search.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Non-random start...

Iteration 1...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -1.8964e+03 -4.8009e+01  6e+04  2e+02  5e-12
 1: -1.4638e+02 -1.8250e+01  3e+03  1e+01  6e-12
 2: -1.4666e+01 -3.8201e+00  2e+02  9e-01  6e-13
 3: -1.3632e+00 -2.7506e+00  1e+01  4e-02  5e-14
 4: -7.2323e-01 -2.0679e+00  1e+00  1e-15  6e-15
 5: -8.1162e-01 -1.0565e+00  2e-01  1e-16  4e-15
 6: -8.5281e-01 -9.1777e-01  6e-02  4e-16  4e-15
 7: -8.6211e-01 -8.9085e-01  3e-02  1e-16  4e-15
 8: -8.6675e-01 -8.8057e-01  1e-02  4e-16  3e-15
 9: -8.6841e-01 -8.7738e-01  9e-03  4e-16  3e-15
10: -8.6997e-01 -8.7462e-01  5e-03  2e-16  3e-15
11: -8.7062e-01 -8.7355e-01  3e-03  6e-16  3e-15
12: -8.7130e-01 -8.7252e-01  1e-03  3e-16  4e-15
13: -8.7161e-01 -8.7208e-01  5e-04  2e-16  4e-15
14: -8.7170e-01 -8.7196e-01  3e-04  6e-16  3e-15
15: -8.7177e-01 -8.7188e-01  1e-04  2e-16  3e-15
16: -8.7179e-01 -8.7184e-01  4e-05  8e-17  3e-15
17: -8.

 2: -1.6368e+00 -2.4751e+00  8e+01  3e-01  1e-13
 3: -2.5149e-01 -2.1181e+00  5e+00  1e-02  8e-15
 4: -1.6820e-01 -8.9065e-01  1e+00  2e-03  1e-14
 5: -1.5088e-01 -4.4541e-01  3e-01  4e-04  5e-15
 6: -1.5544e-01 -2.1691e-01  6e-02  7e-05  2e-15
 7: -1.5656e-01 -2.1549e-01  6e-02  6e-05  1e-15
 8: -1.5966e-01 -1.9227e-01  3e-02  3e-05  9e-16
 9: -1.6287e-01 -1.7575e-01  1e-02  6e-06  9e-16
10: -1.6435e-01 -1.6995e-01  6e-03  2e-06  9e-16
11: -1.6500e-01 -1.6789e-01  3e-03  1e-06  9e-16
12: -1.6533e-01 -1.6696e-01  2e-03  5e-07  8e-16
13: -1.6552e-01 -1.6650e-01  1e-03  3e-07  8e-16
14: -1.6564e-01 -1.6621e-01  6e-04  1e-07  8e-16
15: -1.6578e-01 -1.6593e-01  2e-04  3e-08  1e-15
16: -1.6582e-01 -1.6586e-01  4e-05  7e-09  1e-15
17: -1.6582e-01 -1.6584e-01  2e-05  1e-09  1e-15
18: -1.6583e-01 -1.6584e-01  4e-06  2e-10  1e-15
19: -1.6583e-01 -1.6583e-01  2e-06  2e-11  1e-15
20: -1.6583e-01 -1.6583e-01  3e-07  3e-12  1e-15
21: -1.6583e-01 -1.6583e-01  1e-08  9e-14  1e-15
Optimal solution fou

Recomputing classes...
Class Changes: 14

Iteration 14...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -3.1337e+01 -2.0358e+00  5e+04  2e+02  9e-14
 1: -3.7977e-01 -2.0145e+00  6e+02  3e+00  9e-14
 2: -1.2289e-02 -1.9807e+00  1e+01  4e-02  3e-15
 3: -5.8934e-03 -1.0082e+00  1e+00  2e-03  1e-15
 4: -5.2549e-03 -3.1804e-02  3e-02  1e-05  3e-15
 5: -5.2555e-03 -5.7312e-03  5e-04  1e-07  2e-15
 6: -5.2573e-03 -5.3029e-03  5e-05  1e-08  3e-16
 7: -5.2583e-03 -5.2736e-03  2e-05  3e-09  1e-16
 8: -5.2588e-03 -5.2740e-03  2e-05  3e-09  1e-16
 9: -5.2594e-03 -5.2634e-03  4e-06  5e-10  1e-16
10: -5.2595e-03 -5.2623e-03  3e-06  2e-10  1e-16
11: -5.2596e-03 -5.2607e-03  1e-06  3e-11  1e-16
12: -5.2596e-03 -5.2607e-03  1e-06  3e-11  1e-16
13: -5.2597e-03 -5.2603e-03  6e-07  1e-11  1e-16
14: -5.2597e-03 -5.2600e-03  3e-07  2e-18  2e-16
15: -5.2597e-03 -5.2599e-03  2e-07  3e-18  1e-16
16: -5.2597e-03 -5.2598e-03  9e-08  2e-18  2e-16
Optimal solution found.
Recomputing classes..

     pcost       dcost       gap    pres   dres
 0: -3.1252e+01 -2.0609e+00  5e+04  2e+02  1e-13
 1: -4.1994e-01 -2.0154e+00  6e+02  3e+00  1e-13
 2: -1.2766e-02 -1.9814e+00  1e+01  4e-02  4e-15
 3: -5.9379e-03 -1.0361e+00  1e+00  3e-03  1e-15
 4: -5.2593e-03 -4.1444e-02  4e-02  3e-05  3e-15
 5: -5.2554e-03 -6.4049e-03  1e-03  3e-07  2e-15
 6: -5.2570e-03 -5.3128e-03  6e-05  1e-08  5e-16
 7: -5.2584e-03 -5.2708e-03  1e-05  2e-09  1e-16
 8: -5.2594e-03 -5.2715e-03  1e-05  7e-10  2e-16
 9: -5.2595e-03 -5.2626e-03  3e-06  2e-10  2e-16
10: -5.2596e-03 -5.2624e-03  3e-06  8e-11  1e-16
11: -5.2597e-03 -5.2603e-03  6e-07  1e-11  1e-16
12: -5.2597e-03 -5.2602e-03  5e-07  5e-12  1e-16
13: -5.2597e-03 -5.2601e-03  5e-07  1e-12  1e-16
14: -5.2597e-03 -5.2601e-03  4e-07  1e-12  1e-16
15: -5.2597e-03 -5.2600e-03  3e-07  9e-13  1e-16
16: -5.2597e-03 -5.2599e-03  2e-07  4e-14  1e-16
17: -5.2597e-03 -5.2599e-03  2e-07  4e-14  2e-16
18: -5.2597e-03 -5.2598e-03  1e-07  1e-14  1e-16
19: -5.2597e-03 -5.25

15: -5.2597e-03 -5.2598e-03  2e-07  2e-18  1e-16
16: -5.2597e-03 -5.2597e-03  6e-08  2e-18  1e-16
Optimal solution found.
Recomputing classes...
Class Changes: 22

Iteration 30...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -3.1282e+01 -2.0537e+00  5e+04  2e+02  8e-14
 1: -4.0980e-01 -2.0153e+00  6e+02  3e+00  9e-14
 2: -1.2666e-02 -1.9814e+00  1e+01  4e-02  3e-15
 3: -5.9306e-03 -1.0315e+00  1e+00  3e-03  1e-15
 4: -5.2578e-03 -3.8804e-02  4e-02  2e-05  4e-15
 5: -5.2554e-03 -6.1964e-03  9e-04  2e-07  2e-15
 6: -5.2571e-03 -5.3105e-03  5e-05  1e-08  3e-16
 7: -5.2584e-03 -5.2710e-03  1e-05  2e-09  1e-16
 8: -5.2594e-03 -5.2719e-03  1e-05  8e-10  1e-16
 9: -5.2595e-03 -5.2626e-03  3e-06  2e-10  2e-16
10: -5.2596e-03 -5.2625e-03  3e-06  8e-11  1e-16
11: -5.2597e-03 -5.2603e-03  6e-07  1e-11  1e-16
12: -5.2597e-03 -5.2602e-03  5e-07  3e-12  1e-16
13: -5.2597e-03 -5.2601e-03  4e-07  6e-13  1e-16
14: -5.2597e-03 -5.2600e-03  3e-07  4e-13  1e-16
15: -5.2597e-03 -5.26

15: -5.2597e-03 -5.2598e-03  2e-07  2e-18  1e-16
16: -5.2597e-03 -5.2597e-03  6e-08  2e-18  1e-16
Optimal solution found.
Recomputing classes...
Class Changes: 22

Iteration 38...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -3.1282e+01 -2.0537e+00  5e+04  2e+02  8e-14
 1: -4.0980e-01 -2.0153e+00  6e+02  3e+00  9e-14
 2: -1.2666e-02 -1.9814e+00  1e+01  4e-02  3e-15
 3: -5.9306e-03 -1.0315e+00  1e+00  3e-03  1e-15
 4: -5.2578e-03 -3.8804e-02  4e-02  2e-05  4e-15
 5: -5.2554e-03 -6.1964e-03  9e-04  2e-07  2e-15
 6: -5.2571e-03 -5.3105e-03  5e-05  1e-08  3e-16
 7: -5.2584e-03 -5.2710e-03  1e-05  2e-09  1e-16
 8: -5.2594e-03 -5.2719e-03  1e-05  8e-10  1e-16
 9: -5.2595e-03 -5.2626e-03  3e-06  2e-10  2e-16
10: -5.2596e-03 -5.2625e-03  3e-06  8e-11  1e-16
11: -5.2597e-03 -5.2603e-03  6e-07  1e-11  1e-16
12: -5.2597e-03 -5.2602e-03  5e-07  3e-12  1e-16
13: -5.2597e-03 -5.2601e-03  4e-07  6e-13  1e-16
14: -5.2597e-03 -5.2600e-03  3e-07  4e-13  1e-16
15: -5.2597e-03 -5.26

15: -5.2597e-03 -5.2598e-03  2e-07  2e-18  1e-16
16: -5.2597e-03 -5.2597e-03  6e-08  2e-18  1e-16
Optimal solution found.
Recomputing classes...
Class Changes: 22

Iteration 46...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -3.1282e+01 -2.0537e+00  5e+04  2e+02  8e-14
 1: -4.0980e-01 -2.0153e+00  6e+02  3e+00  9e-14
 2: -1.2666e-02 -1.9814e+00  1e+01  4e-02  3e-15
 3: -5.9306e-03 -1.0315e+00  1e+00  3e-03  1e-15
 4: -5.2578e-03 -3.8804e-02  4e-02  2e-05  4e-15
 5: -5.2554e-03 -6.1964e-03  9e-04  2e-07  2e-15
 6: -5.2571e-03 -5.3105e-03  5e-05  1e-08  3e-16
 7: -5.2584e-03 -5.2710e-03  1e-05  2e-09  1e-16
 8: -5.2594e-03 -5.2719e-03  1e-05  8e-10  1e-16
 9: -5.2595e-03 -5.2626e-03  3e-06  2e-10  2e-16
10: -5.2596e-03 -5.2625e-03  3e-06  8e-11  1e-16
11: -5.2597e-03 -5.2603e-03  6e-07  1e-11  1e-16
12: -5.2597e-03 -5.2602e-03  5e-07  3e-12  1e-16
13: -5.2597e-03 -5.2601e-03  4e-07  6e-13  1e-16
14: -5.2597e-03 -5.2600e-03  3e-07  4e-13  1e-16
15: -5.2597e-03 -5.26

ValueError: Can't handle mix of binary and continuous